In [19]:
import java.io.File
import util.InputReader

typealias PuzzleLine = String
typealias PuzzleInput = List<PuzzleLine>

val exampleInput: PuzzleInput = InputReader.getExampleLines(2023, 24)
val puzzleInput: PuzzleInput = InputReader.getPuzzleLines(2023, 24)

In [20]:
exampleInput

[19, 13, 30 @ -2,  1, -2, 18, 19, 22 @ -1, -1, -2, 20, 25, 34 @ -2, -2, -4, 12, 31, 28 @ -1, -2, -1, 20, 19, 15 @  1, -5, -3]

In [21]:
data class Hailstone2d(val px: Long, val py: Long, val vx: Long, val vy: Long)

fun from(string: String): Hailstone2d {
    val positions = string.split(" @ ").flatMap { it.split(", ") }.map { it.trim().toLong() }
    return Hailstone2d(positions[0], positions[1], positions[3], positions[4])
}

In [22]:
from(exampleInput[0])

Hailstone2d(px=19, py=13, vx=-2, vy=1)

In [23]:
fun List<String>.toHailstones(): List<Hailstone2d> = this.map { from(it) }

In [24]:
exampleInput.toHailstones()

[Hailstone2d(px=19, py=13, vx=-2, vy=1), Hailstone2d(px=18, py=19, vx=-1, vy=-1), Hailstone2d(px=20, py=25, vx=-2, vy=-2), Hailstone2d(px=12, py=31, vx=-1, vy=-2), Hailstone2d(px=20, py=19, vx=1, vy=-5)]

In [25]:
val a = exampleInput.toHailstones()[0]
val b = exampleInput.toHailstones()[1]
listOf(a,b)

[Hailstone2d(px=19, py=13, vx=-2, vy=1), Hailstone2d(px=18, py=19, vx=-1, vy=-1)]

In [26]:
inline operator fun Hailstone2d.plus(other: Hailstone2d): Hailstone2d {
    return Hailstone2d(px + other.px, py + other.py, vx + other.vx, vy + other.vy)
}

inline operator fun Hailstone2d.minus(other: Hailstone2d): Hailstone2d {
    return Hailstone2d(px - other.px, py - other.py, vx - other.vx, vy - other.vy)
}


b - a

Hailstone2d(px=-1, py=6, vx=1, vy=-2)

In [27]:
// y = mx + c
val Hailstone2d.m: Double
    get() = vy.toDouble() / vx.toDouble()

val Hailstone2d.c: Double
    get() = py.toDouble() - m * px.toDouble()

fun intersectX(a: Hailstone2d, b: Hailstone2d): Double {
    return (b.c - a.c) / (a.m - b.m)
}


In [28]:
intersectX(a, b)

14.333333333333334

In [29]:
fun intersectY(a: Hailstone2d, b: Hailstone2d): Double {
    return a.m * intersectX(a, b) + a.c
}

In [30]:
intersectY(a, b)

15.333333333333332

In [40]:
data class Box(val x: LongRange, val y: LongRange) {
    constructor(x: IntRange, y: IntRange) : this(x.toLongRange(), y.toLongRange())
    constructor(xy: IntRange) : this(xy, xy)
    constructor(start: String, end: String) : this(start.toLong(), end.toLong())
    constructor(start: Long, end: Long) : this(start..end, start..end)
}

fun IntRange.toLongRange(): LongRange = this.first.toLong()..this.last.toLong()

fun Box.xInside(x: Double): Boolean = x >= this.x.first.toDouble() && x <= this.x.last.toDouble()
fun Box.yInside(y: Double): Boolean = y >= this.y.first.toDouble() && y <= this.y.last.toDouble()
fun Box.inside(x: Double, y: Double): Boolean = xInside(x) && yInside(y)

fun Box.intersectionInside(a: Hailstone2d, b: Hailstone2d): Boolean {
    val x = intersectX(a, b)
    val y = intersectY(a, b)
    return inside(x, y)
}

In [41]:
fun <E>Collection<E>.combinations(): List<Pair<E, E>> {
    val result = mutableListOf<Pair<E, E>>()
    for (i in this.indices) {
        for (j in i + 1 until this.size) {
            val pair = this.elementAt(i) to this.elementAt(j)
            result.add(pair)
        }
    }
    return result
}

In [42]:
data class Vector2d(val x: Double, val y: Double) {
    fun lengthSquared(): Double = x * x + y * y

    operator fun plus(other: Vector2d) = Vector2d(x + other.x, y + other.y)
    operator fun minus(other: Vector2d) = Vector2d(x - other.x, y - other.y)
}


val Hailstone2d.position: Vector2d
    get() = Vector2d(px.toDouble(), py.toDouble())

val Hailstone2d.velocity: Vector2d
    get() = Vector2d(vx.toDouble(), vy.toDouble())


fun Hailstone2d.distance(other: Hailstone2d): Double {
    return (this.position - other.position).lengthSquared()
}

fun Hailstone2d.tick() = copy(px = px + vx, py = py + vy)

val Double.unit: Double get() = if (this > 0) 1.0 else if (this < 0) -1.0 else 0.0
val Vector2d.unit: Vector2d get() = Vector2d(x = x.unit, y = y.unit)

fun Hailstone2d.travellingTowards(other: Vector2d): Boolean {
    return (this.velocity.unit == (other - this.position).unit)
}


fun converging(a: Hailstone2d, b: Hailstone2d): Boolean {
    if (a.m == b.m) return false
    
    val intersection = Vector2d(intersectX(a, b), intersectY(a, b))
    val aCorrect = a.travellingTowards(intersection)
    val bCorrect = b.travellingTowards(intersection)
    return aCorrect && bCorrect
}

Hailstone A: 19, 13, 30 @ -2, 1, -2
Hailstone B: 20, 19, 15 @ 1, -5, -3
Hailstones' paths crossed in the past for hailstone A.

In [43]:
val a = Hailstone2d(19, 13, -2, 1)
val b = Hailstone2d(20, 19, 1, -5)
converging(a, b)

false

In [44]:
fun partOne(input: List<String>, box: Box): Int {
    val hailstones = input.toHailstones()
    val combinations = hailstones.combinations()
    val intersectsInside = combinations.filter { (a, b) -> converging(a, b) && box.intersectionInside(a, b) }
    return intersectsInside.size
}

partOne(exampleInput, Box(7..27))

2

In [45]:
partOne(puzzleInput, Box("200000000000000", "400000000000000"))

18651